In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    GenerationConfig,
    set_seed
)
import torch

/user_data/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
model_name = 'taide_markdown_3-full' # 模型路徑
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto')
model.eval() # 切成inference模式
print('load model complete')

Loading checkpoint shards: 100%|██████████| 3/3 [02:48<00:00, 56.12s/it]


load model complete


In [27]:
# 使用pipeline生成

# question = "問題:\n" + "誰做到純甲醇的首次分離" + "\n\n"
context = "內文:\n" + "甲醇又稱羥基甲烷、木醇與木精，是一種有機化合物，為最簡單的醇類。木醇與木精之名源自於甲醇的主要生產方式曾經是萃取自木醋液。現代甲醇是直接從一氧化碳，二氧化碳和氫的一個催化作用的工業過程中製備。甲醇很輕、揮發度高、無色、易燃及有一個獨特的氣味非常相似乙醇的氣味。但不同於乙醇，甲醇有劇毒，不可以飲用。通常用作溶劑、防凍劑、燃料或變性劑，亦可用於經過酯交換反應生產生物柴油。甲醇的火焰近乎無色，所以燃點甲醇時要格外小心，以免被燒傷。不少細菌在進行缺氧新陳代謝之時，都會產生甲醇。因此，空氣中存有少量甲醇的蒸氣，但幾日內就會在陽光照射之下被空氣中的氧氣氧化，成為二氧化碳。古埃及人在遺體保存技術防腐的過程中，使用了包括甲醇的混合物質以保存屍體。然而，純甲醇的首次分離於1661年由羅伯特·波義耳，當他產生它通過蒸餾黃楊。它後來被稱為pyroxylic spirit。 1834年，法國化學家讓-巴蒂斯特·杜馬和尤金·皮里哥確定了它元素組成。在2006年，天文學家利用射電望遠鏡在焦德雷爾班克天文台的MERLIN陣列發現了太空中一個2880億英里寬的巨大甲醇雲。甲醇最大的用途是製造其他化學品。大約40%的甲醇會被轉化為甲醛，再用於生產塑料，膠合板，塗料，炸藥, 和免燙紡織品。甲醇亦有時會用於更高的分子建構，例如：利用改性高嶺土作催化劑，可以把甲醇脫水生成二甲醚。科學家基於高嶺土在類似的環境所扮演的催化角色，以及地球創始之時的豐富甲烷環境，推論出地球生命的生成可能從甲醇開始。" + "\n\n</s>"

user_message = context
system_prompt = '<<SYS>>\n\n請根據以下提供的文章，生成出一個問題\n<</SYS>>\n'

generation_params = {
    "max_new_tokens": 10,
    "top_k": 50,
    "num_beams": 4,  # 控制beam search的寬度
    "no_repeat_ngram_size": 2,  # 控制生成文本中不重複的n-gram大小
}

pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=2048)
result = pipe(f"[INST] {system_prompt}\n{user_message} [/INST]", **generation_params)
print(result[0]['generated_text'])

Both `max_new_tokens` (=10) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[INST] <<SYS>>

請根據以下提供的文章，生成出一個問題
<</SYS>>

內文:
甲醇又稱羥基甲烷、木醇與木精，是一種有機化合物，為最簡單的醇類。木醇與木精之名源自於甲醇的主要生產方式曾經是萃取自木醋液。現代甲醇是直接從一氧化碳，二氧化碳和氫的一個催化作用的工業過程中製備。甲醇很輕、揮發度高、無色、易燃及有一個獨特的氣味非常相似乙醇的氣味。但不同於乙醇，甲醇有劇毒，不可以飲用。通常用作溶劑、防凍劑、燃料或變性劑，亦可用於經過酯交換反應生產生物柴油。甲醇的火焰近乎無色，所以燃點甲醇時要格外小心，以免被燒傷。不少細菌在進行缺氧新陳代謝之時，都會產生甲醇。因此，空氣中存有少量甲醇的蒸氣，但幾日內就會在陽光照射之下被空氣中的氧氣氧化，成為二氧化碳。古埃及人在遺體保存技術防腐的過程中，使用了包括甲醇的混合物質以保存屍體。然而，純甲醇的首次分離於1661年由羅伯特·波義耳，當他產生它通過蒸餾黃楊。它後來被稱為pyroxylic spirit。 1834年，法國化學家讓-巴蒂斯特·杜馬和尤金·皮里哥確定了它元素組成。在2006年，天文學家利用射電望遠鏡在焦德雷爾班克天文台的MERLIN陣列發現了太空中一個2880億英里寬的巨大甲醇雲。甲醇最大的用途是製造其他化學品。大約40%的甲醇會被轉化為甲醛，再用於生產塑料，膠合板，塗料，炸藥, 和免燙紡織品。甲醇亦有時會用於更高的分子建構，例如：利用改性高嶺土作催化劑，可以把甲醇脫水生成二甲醚。科學家基於高嶺土在類似的環境所扮演的催化角色，以及地球創始之時的豐富甲烷環境，推論出地球生命的生成可能從甲醇開始。

<\s> [/INST] Here is a question based on the provided article:


In [4]:
# 使用generate()生成

context = "參考文章:\n" + "防治方法：\n五、任選以下一種藥劑於發生茶叢行局部施藥，幼齡期防治效果最佳。\n|藥劑名稱 | 每公頃使用量 | 稀釋倍數(倍) | 施藥方法 | 注意事項\n|:----------------------|:------------|:-----------|:-----------|:---------------------------------------------------|\n10%美文松乳劑(Mevinphos) | 2.6公升 | 400 | 害蟲發生時施藥。 | 採收前7天停止施藥。\n10%美文松溶液(Mevinphos) | 2.6公升 | 400 | 害蟲發生時施藥。 | 採收前7天停止施藥。\n2.8%第滅寧乳劑(Deltamethrin) | 0.35公升 | 3,000 | 害蟲發生時施藥。 | 1.防治圖紋尺蠖蛾。2.採收前10天停止施藥。\n2.8%第滅寧水基乳劑(Deltamethrin) | 0.35公升 | 3,000 | 害蟲發生時施藥。 | 1.防治圖紋尺蠖蛾。2.採收前10天停止施藥。\n95%三氯松水溶性粉劑(Trichlorfon) | 1.7公斤 | 720 | 害蟲發生時施藥。 | 採收前14天停止施藥。" + "\n\n</s>"
question = "問題:\n" + "使用第滅寧乳劑施藥時，稀釋倍數是多少？"

user_message = context
system_prompt = '<<SYS>>\n\n你是一位農業病蟲害的專家，請根據提供的參考文章中尋找答案並回答\n<</SYS>>\n'

inputs = tokenizer(f"[INST] {system_prompt}\n{user_message} [/INST]",add_special_tokens=False, return_tensors="pt").to(device)
# input_len = inputs.input_ids.size(1)

generation_config = GenerationConfig(
    max_new_tokens=1000, # 控制生成文本的最大新增token數
    # max_length=1000, # 控制生成答案的最大長度(效果會被max_new_token蓋掉)
    top_k=50, # 限制考慮的詞彙數量
    num_beams=2, # 控制beam search的寬度
    num_return_sequences=1, # 生成的答案數量
    # no_repeat_ngram_size=5, # 限制不重複的n-gram大小
    early_stopping=True,
)

# set_seed(42)
# generation_config = GenerationConfig(
#     max_new_tokens=1000,
#     do_sample = True,
#     top_k=5000,
#     top_p=0.95,
#     temperature=0.8,
# )

outputs = model.generate(
    **inputs,
    generation_config=generation_config
)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


[INST] <<SYS>>

請根據以下提供的文章，生成出一個問題
<</SYS>>

內文:
甲醇又稱羥基甲烷、木醇與木精，是一種有機化合物，為最簡單的醇類。木醇與木精之名源自於甲醇的主要生產方式曾經是萃取自木醋液。現代甲醇是直接從一氧化碳，二氧化碳和氫的一個催化作用的工業過程中製備。甲醇很輕、揮發度高、無色、易燃及有一個獨特的氣味非常相似乙醇的氣味。但不同於乙醇，甲醇有劇毒，不可以飲用。通常用作溶劑、防凍劑、燃料或變性劑，亦可用於經過酯交換反應生產生物柴油。甲醇的火焰近乎無色，所以燃點甲醇時要格外小心，以免被燒傷。不少細菌在進行缺氧新陳代謝之時，都會產生甲醇。因此，空氣中存有少量甲醇的蒸氣，但幾日內就會在陽光照射之下被空氣中的氧氣氧化，成為二氧化碳。古埃及人在遺體保存技術防腐的過程中，使用了包括甲醇的混合物質以保存屍體。然而，純甲醇的首次分離於1661年由羅伯特·波義耳，當他產生它通過蒸餾黃楊。它後來被稱為pyroxylic spirit。 1834年，法國化學家讓-巴蒂斯特·杜馬和尤金·皮里哥確定了它元素組成。在2006年，天文學家利用射電望遠鏡在焦德雷爾班克天文台的MERLIN陣列發現了太空中一個2880億英里寬的巨大甲醇雲。甲醇最大的用途是製造其他化學品。大約40%的甲醇會被轉化為甲醛，再用於生產塑料，膠合板，塗料，炸藥, 和免燙紡織品。甲醇亦有時會用於更高的分子建構，例如：利用改性高嶺土作催化劑，可以把甲醇脫水生成二甲醚。科學家基於高嶺土在類似的環境所扮演的催化角色，以及地球創始之時的豐富甲烷環境，推論出地球生命的生成可能從甲醇開始。

</s> [/INST] 催化劑、甲醛、塑料、膠合板、塗料、炸藥、紡織品、二甲醚、高嶺土、催化作用、甲烷、生命起源。
